In [1]:
#!/usr/bin/env python3
import os
import numpy as np
import cv2
###-------------- LSTM sequence generator ----------------------------------------------------------------------------------------------------------------------------------------################################
###-------------- Input file: Lidar_Images_Sim'-----------------------------------------------------------------------------------------------------------------------------------################################
###-------------- Output file: save files in Images_Tensor_Sim, train and test in one file.---------------------------------------------------------------------------------------################################
SIMULATION=0
sequence_len = 5
ModelCar_Path = os.path.dirname(os.path.abspath(''))
Data_Path = os.path.join(ModelCar_Path, 'Data')
if(SIMULATION):
    Image_Path = os.path.join(Data_Path, 'Images_Sim')
    Train_Image_Path = os.path.join(Image_Path, 'Train_Images_Sim')
    Test_Image_Path = os.path.join(Image_Path, 'Test_Images_Sim')
    Whole_Image_Path = os.path.join(Image_Path, 'Lidar_Images_Sim')
    Manual_Path = os.path.join(Data_Path, 'Manual_Sim.npy')
    Edge_Camera_Path = os.path.join(Data_Path, 'Edge_Camera_Sim.npy')
    Nvidia_Camera_Path = os.path.join(Data_Path, 'Nvidia_Camera_Sim.npy')
    Lidar_Path = os.path.join(Data_Path, 'Lidar_Sim.npy')   
    Image_Save_Path = os.path.join(Image_Path, 'Images_Tensor_Sim')
else:
    Image_Path = os.path.join(Data_Path, 'Images_Car')
    Train_Image_Path = os.path.join(Image_Path, 'Train_Images_Car')
    Test_Image_Path = os.path.join(Image_Path, 'Test_Images_Car')
    Whole_Image_Path = os.path.join(Image_Path, 'EECE_Images')
    #Whole_Image_Path = '/media/zhihuilai/RAID1/Erik-ModelCar-Data/CME_Images_Processed'
    Manual_Path = os.path.join(Data_Path, 'Manual_Car.npy')
    Edge_Camera_Path = os.path.join(Data_Path, 'Edge_Camera_Car.npy')
    Nvidia_Camera_Path = os.path.join(Data_Path, 'Nvidia_Camera_Car.npy')
    Lidar_Path = os.path.join(Data_Path, 'Lidar_Car.npy')
    #Image_Save_Path = os.path.join(Image_Path, 'CME_Tensors')
    #Image_Save_Path = "/media/zhihuilai/RAID1/Erik-ModelCar-Data/CME_Tensors_Processed"
    Image_Save_Path = "/media/zhihuilai/RAID1/Erik-ModelCar-Data/EECE_Tensors_Processed"
if SIMULATION:
    def Image_Processing(image):
        image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        #image = cv2.GaussianBlur(image, (3, 3), 0)
        image = cv2.resize(image, (230, 66))
        return image       
else:   #reality
    def Image_Processing(image):
        image = image[60:165, :, :]
        #105,320
        #105,280
        image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        #image = cv2.GaussianBlur(image, (3, 3), 0)
        image = cv2.resize(image, (230, 66))
        return image
Images_sequence_array = []
sequence = []
Image_Count=0
if(len(os.listdir(Whole_Image_Path)) == 0):
    print('[Error!] Check Whole Images are in Correct Directory')
    sys.exit()

for Filepath in os.listdir(Whole_Image_Path):
    Search_Folder = os.path.join(Whole_Image_Path, Filepath)
    sequence = []
    for Folders in sorted(os.listdir(Search_Folder), key=lambda x: int(x.split("_")[2])):
        #add manual images 
        #if (int(Folders.split("_")[2])%2)==1:
        Whole_Images_Array = np.load(os.path.join(Search_Folder, Folders), allow_pickle=True)
        sequence.append(Image_Processing(Whole_Images_Array[0]))
        #sequence.append(Whole_Images_Array[0])
        if (len(sequence) == sequence_len):
            sequence_arr=np.array(sequence)
            Speed = Whole_Images_Array[1]
            Steering_Angle = Whole_Images_Array[2]
            Images_sequence_array = [sequence_arr, Speed, Steering_Angle]
            Image_Filename = 'Image_Tensor_' + str(sequence_len)+'_'+str(Image_Count) + '_' + str(Speed) + '_' + str(Steering_Angle)
            np.save(os.path.join(Image_Save_Path, Image_Filename), np.array(Images_sequence_array))
            Image_Count += 1
            sequence.pop(0)  
              
#print("Image shape:",Images_arr.shape)

In [ ]:
#!/usr/bin/env python3
import os
import numpy as np
CNN_Type = 0
#0 = NVIDIA 
#1 = Features
Version = 'EECE'

Train_Model = 1
SIMULATION = 1
Use_Speed = 1
Show_Results = 1
sequence_len = 5
file_count=0
ModelCar_Path = os.path.dirname(os.path.abspath(''))
Data_Path = os.path.join(ModelCar_Path, 'Data')
Training_Results_Path = os.path.join(Data_Path, Version)
Model_Path = os.path.join(Training_Results_Path, Version + '.h5')
Check_Path = os.path.join(Training_Results_Path, 'Model_Check.h5')
History_Path = os.path.join(Training_Results_Path, 'History.pickle')
Prediction_Path = os.path.join(Training_Results_Path, 'Predictions')
Image_Path = os.path.join(Data_Path, 'Images_Sim')
Train_Image_Path = os.path.join(Image_Path, 'Train_Images_Sim')
Test_Image_Path = os.path.join(Image_Path, 'Test_Images_Sim')
Manual_Path = os.path.join(Data_Path, 'Manual_Sim.npy')
Edge_Camera_Path = os.path.join(Data_Path, 'Edge_Camera_Sim.npy')
Nvidia_Camera_Path = os.path.join(Data_Path, 'Nvidia_Camera_Sim.npy')
Lidar_Path = os.path.join(Data_Path, 'Lidar_Sim.npy')
Images = []
Speeds = []
Steering_Angles = []
if(len(os.listdir(Train_Image_Path)) == 0):
    print('[Error!] Check Training Images are in Correct Directory')
    sys.exit()
'''
#filename sequence debugging
for Filepath in os.listdir(Train_Image_Path):
    Search_Folder = os.path.join(Train_Image_Path, Filepath)
    sequence = []
    #Images_arr=np.asarray(Images)
    #print("Image shape:", Images_arr.shape)
    for Train_Folders in sorted(os.listdir(Search_Folder), key=lambda x: int(x.split("_")[2])):
        Training_Images_Array = np.load(os.path.join(Search_Folder, Train_Folders), allow_pickle=True)
        #print(os.path.basename(os.path.join(Search_Folder, Train_Folders)))
        #sequence.append(Training_Images_Array[0])
        sequence.append(os.path.basename(os.path.join(Search_Folder, Train_Folders)))
        #print(sequence)
        if (len(sequence) == sequence_len):
            #Images.append(sequence)
            Images=np.append(Images,np.asarray(sequence))
            #Images=np.expand_dims(Images,axis=1)
            #sequence_arr=np.asarray(sequence)
            #Images_arr=np.asarray(Images)
            #print("sequence shape: ", sequence_arr.shape)
            #print("Image shape:", Images_arr.shape)
            Speeds.append(Training_Images_Array[1])
            Steering_Angles.append(Training_Images_Array[2])
            sequence.pop(0)
Images=np.resize(Images,(int(Images.size/5),5))
Images.shape
'''
for Filepath in os.listdir(Train_Image_Path):
    Search_Folder = os.path.join(Train_Image_Path, Filepath)
    sequence = []
    for Train_Folders in sorted(os.listdir(Search_Folder), key=lambda x: int(x.split("_")[2])):
        Training_Images_Array = np.load(os.path.join(Search_Folder, Train_Folders), allow_pickle=True)
        sequence.append(Training_Images_Array[0])
        file_count=file_count+1
        #print(file_count)
        #Images_arr=np.array(Images)
        #print("Image shape:", Images_arr.shape)
        if (len(sequence) == sequence_len):
            '''
            if(file_count==sequence_len):#initialization
                #Images.append(sequence)
                sequence_arr=np.expand_dims(np.asarray(sequence),axis=0)
                Images=sequence_arr
                print(Images.shape)
            else:
                sequence_arr=np.expand_dims(np.asarray(sequence),axis=0)
                Images=np.append(Images,sequence_arr,axis=0)
            '''
            #Images.extend(sequence)
            sequence_arr=np.array(sequence)
            Images.append(sequence_arr)
            Speeds.append(Training_Images_Array[1])
            Steering_Angles.append(Training_Images_Array[2])
            sequence.pop(0)        
#Images_arr=np.asarray(Images)
#print("Image pre shape:", Images_arr.shape)
#Images_arr=np.reshape(np.asarray(Images),(int(len(Images)/sequence_len),sequence_len, 240, 320, 3))
print("Image shape:",Images_arr.shape)

In [3]:
import numpy as np
a=[0,1,2,3,4]
a.append(5)
a.pop(0)
a.append(6)
a.pop(0)
b=[3,4,5,6,7]
c=[]
c.append(a)
c.append(b)
d=np.asarray(c)
print(d.shape)

(2, 5)


In [14]:
sequence_arr=np.expand_dims(np.asarray(sequence),axis=0)
print(sequence_arr.shape)
#abc=np.expand_dims(np.asarray(sequence),axis=1)
#abc.shape
abc=[]
abc=sequence_arr
abc=np.append(abc,sequence_arr,axis=0)
sequence_arr=np.expand_dims(np.asarray(sequence),axis=0)
abc=np.append(abc,sequence_arr,axis=0)
abc.shape

(1, 5, 240, 320, 3)


(3, 5, 240, 320, 3)

In [11]:
#Images_arr=np.array(Images_sequence_array)
#Images_arr=np.array(Whole_Images_Array)
#print("Image shape:",Images_arr[0].shape)
print("Image shape:",Images_sequence_array[0].shape)
#Images_arr[0]

Image shape: (5, 240, 320, 3)


In [2]:
Images_arr=np.asarray(Images)
print("Image pre shape:", Images_arr.shape)
Images_arr=np.reshape(Images_arr,(int(np.size(Images_arr,0)/sequence_len),sequence_len, 240, 320, 3))
#Images_arr.size
print("Image shape:",Images_arr.shape)
#print("Image size:",Images_arr.size)
#Images

In [ ]:
print(np.size(Images_arr,0))

Error: Kernel is dead

In [1]:
import os
import numpy as np
ModelCar_Path = os.path.dirname(os.path.abspath(''))
Data_Path = os.path.join(ModelCar_Path, 'Data')
Image_Path = os.path.join(Data_Path, 'Images_Sim')
Image_Save_Path = os.path.join(Image_Path, 'Images_Tensor_Sim')
sequence_len = 5
import cv2 
#for tensor in sorted(os.listdir(Image_Save_Path), key=lambda x: int(x.split("_")[3])):
for tensor in os.listdir(Image_Save_Path):
    img_array = np.load(os.path.join(Image_Save_Path, tensor), allow_pickle=True)
    for j in range (sequence_len):
        cv2.imshow('Record',img_array[0][j])
        #print("i: ",i,"j: ",j)
        key=cv2.waitKey()
        if key==27: break
    if key==27: break
cv2.destroyAllWindows()


In [2]:
import os
import numpy as np
import cv2 
Images=[]
Speeds=[]
Steering_Angles=[]
#sequence_trimmed=[]
Images_trimmed=[]
sequence_len=5
ModelCar_Path = os.path.dirname(os.path.abspath(''))
Data_Path = os.path.join(ModelCar_Path, 'Data')
Image_Path = os.path.join(Data_Path, 'Images_Sim')
Image_Save_Path = os.path.join(Image_Path, 'Images_Tensor_Sim')
for tensor in os.listdir(Image_Save_Path):
    img_array = np.load(os.path.join(Image_Save_Path, tensor), allow_pickle=True)
    Images.append(img_array[0])
    #Speeds.append(img_array[1])
    #Steering_Angles.append(img_array[2])
#Images_arr=np.array(Images)
print(len(Images))
#print(Images_arr.shape)

20570


In [11]:
def Image_Processing(image):
    image = image[80:, :, :]
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    image = cv2.GaussianBlur(image, (3, 3), 0)
    image = cv2.resize(image, (200, 66))
    return image
Images_trimmed=[]
for i in range(100):
    sequence_trimmed=[]
    for j in range(sequence_len):
        Image = Image_Processing(Images[i][j])
        sequence_trimmed.append(Image)
    Images_trimmed.append(sequence_trimmed)
        #Speed_Test.append(Test_Speeds[i])
        #Steering_Angle_Test.append(Test_Steering_Angles[i])

In [12]:
import cv2 
for i in range(len(Images_trimmed)):
    for j in range(sequence_len):
        cv2.imshow('Record',Images_trimmed[i][j])
        #print("i: ",i,"j: ",j)
        key=cv2.waitKey()
        if key==27: break
    if key==27: break
cv2.destroyAllWindows()

In [3]:
Images_arr=np.array(Images)
print(Images_arr.shape)

(1329, 5, 240, 320, 3)


In [4]:
import cv2 
img_array =Images_arr
for i in range (0,2000):
    cv2.imshow('Record',img_array[i][0])
    #print("i: ",i,"j: ",j)
    key=cv2.waitKey()
    if key==27: break
cv2.destroyAllWindows()

In [ ]:
sequence_arr=np.asarray(sequence)
print(sequence_arr.shape)
abc=[]
abc.append(sequence)
abc.append(sequence)
abc.append(sequence)
abc_arr=np.asarray(abc)
print(abc_arr.shape)
abc_arr=np.asarray(abc)
print(abc_arr.shape)

Error: Kernel is dead